<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/AdvML2024_ex03notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex03notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)

※ この notebook は，授業時間中の解説や板書と併用することを想定して作っていますので，説明が不十分なところが多々あります．


----
## 準備
----


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
### データを生成する関数
#
def generateData(N, trueFunc=False, seed=None, sigma=0.0):

    if trueFunc:
        x = np.linspace(-0.1, 1.1, num=N)
        y = np.sin(2*np.pi*x) + 1
    else:
        # 乱数生成器
        rng = np.random.default_rng(seed)
        x = rng.uniform(0.0, 1.0, N)
        y = np.sin(2*np.pi*x) + 1 + sigma*rng.standard_normal(N)

    return x, y

In [ ]:
##### 1, x, x^2, x^3, ..., x^D をならべたデータ行列（N x (D+1)）をつくる
#
def makeDataMatrix(x, D):

    N = x.shape[0]
    X = np.zeros((N, D+1))
    X[:, 0] = 1
    for d in range(1, D+1):
        X[:, d] = x**d

    return X

In [ ]:
### 線形回帰
#
def LinearRegression(X, y):
    # 正規方程式の左辺の行列を A とする
    A = X.T @ X
    # 正規方程式の右辺のベクトルを b とする
    b = X.T @ y
    # np.linalg.solve を使って正規方程式を解き，解を w とする
    w = np.linalg.solve(A, b)
    return w

In [ ]:
### リッジ回帰
#
def RidgeRegression(X, y, lam, excludeBias=True):
    # 正規方程式の左辺の行列を A とする
    eyet = np.ones(X.shape[1])
    if excludeBias:
        eyet[0] = 0.0
    A = X.T @ X + lam * np.diag(eyet)
    # 正規方程式の右辺のベクトルを XTY とする
    b = X.T @ y
    #print(X.shape, A.shape, b.shape)
    # np.linalg.solve を使って正規方程式を解き，解を w とする
    w = np.linalg.solve(A, b)
    return w

---
## バイアス vs バリアンス
---

汎化性能の性質について，理論的な側面から解説する．ただし，ここでは厳密な議論を避けて直感的な説明にとどめる．詳しくは↓等を参考にされたい．

【参考】 PRML本

データ $\mathbf{x}$ から $y \in R$ を予測する回帰の問題を考える． $\mathbf{x}$ と $y$ の関係は関数 $h(\mathbf{x})$ で定まるものとする．
様々な $\mathbf{x}$ と $y$ の観測値を集めたデータ集合を ${\cal D}$ と表記し，${\cal D}$ を用いてあるモデルを学習させて得られる関数を $f(\mathbf{x}; {\cal D})$ と表記する．

ここで，${\cal D}$ はひとつに定まるものではなく，無数に存在するとして統計的な取り扱いをすることに注意．例えば，前回の多項式回帰の実験では，学習データのうち入力のサンプルは $[0, 1]$ の一様分布からランダムサンプリングした値だった．出力の正解は，真の値 $h(x)$ に正規乱数を加えた値だった．このような場合，実験のたびに学習に使うデータ集合が変化する．

このような問題設定においては，${\cal D}$ を用いて得られたモデルによる予測値 $f(\mathbf{x}; {\cal D})$ と真の値 $h(\mathbf{x})$ との間の二乗誤差 $ \{f(\mathbf{x}; {\cal D}) - h(\mathbf{x})\}^2$ が，このモデルの汎化性能の指標となる．
この値は ${\cal D}$ の取り方によって変化するので，その期待値 $\textrm{E} \left[ \{f(\mathbf{x}; {\cal D}) - h(\mathbf{x})\}^2 \right]$ のふるまいを調べる．

具体的な計算過程は省略するが，$\textrm{E} \left[ \{f(\mathbf{x}; {\cal D}) - h(\mathbf{x})\}^2 \right]$ は次のように変形できる．

$$
\textrm{E} \left[ \{f(\mathbf{x}; {\cal D}) - h(\mathbf{x})\}^2 \right]
=　\underbrace{\left\{ \textrm{E} \left[ f(\mathbf{x}; {\cal D}) \right] - h(\mathbf{x}) \right\}^2}_{(\text{bias})^2}
+ \underbrace{\textrm{E} \left[ \left\{ f(\mathbf{x}; {\cal D}) - \textrm{E}\left[ f(\mathbf{x}; {\cal D})\right] \right\}^2 \right]}_{\text{variance}}
$$


右辺第1項は **バイアス** (bias) と呼ばれる量の二乗で，取り得るすべての ${\cal D}$ に関する予測値 $f(\mathbf{x};{\cal D})$ の期待値 $\textrm{E} \left[ f(\mathbf{x}; {\cal D}) \right]$ が真の関数 $h(\mathbf{x})$ とどのくらいずれているかを表す量である．

一方，右辺第2項は **バリアンス** (variance) と呼ばれる量で，個々の ${\cal D}$ で得られる $f(\mathbf{x}; \cal{D})$ が，それらの期待値 $\textrm{E}\left[ f(\mathbf{x}; {\cal D})\right]$ のまわりでどのくらい変動するのか，その変動の大きさを表している（注）．
データ集合 ${\cal D}$ の取り方によって $f(\mathbf{x}; {\cal D})$ が大きく変わるならば，この量は大きくなる．

<div stype="font-size:x-small">
注: $X$ に関する「分散」（英語で variance）は $\textrm{E}\left[ (X - \textrm{E}(X))^2 \right]$ と表せるのだった．
</div>




バイアスの2乗もバリアンスも非負だから，予測の二乗誤差の期待値が一定とすると，これらは互いにトレードオフの関係（一方が大きくなると他方が小さくなるような関係）にある．
下図は，そのことを模式的に表したものである．

上段は，複雑度の小さいモデル（例: 多項式回帰で次数が小さいときや正則化パラメータの値が大きいとき）の図で，左に，様々な${\cal D}$ で得られる予測値 $f(x; {\cal D})$ を，右に，それらの期待値 $\textrm{E} \left[ f(x; {\cal D})\right]$ と真の関数 $h(x)$ を描いている．
左図に示すように， $f(x; {\cal D})$ の変動は小さい（バリアンスが小さい）が，右図から分かるように，それらの期待値と真の値とのずれは大きい（バイアスの2乗が大きい）．

一方，複雑度の大きいモデル（下段）の場合，個々のモデルがデータによく当てはまり過ぎるため，$f(x; {\cal D})$ の変動が大きく（バリアンスが大きく）なっているが，それらの期待値と真の値とのずれは小さく（バイアスの2乗が小さく）なっている．

モデルの複雑度が小さすぎる場合，試行ごとの予測値のばらつきは小さいものの，どれもデータにうまく当てはまらない．逆に，複雑度が大きすぎる場合，個々のモデルがデータによく当てはまりすぎて，試行ごとの予測値のばらつきが大きくなる．汎化性能の観点からは，バイアスとバリアンスの双方がバランスよく小さくなるようなモデルが望ましい．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/biasvsvariance.png">

---
## モデルの検証と選択
---

---
### 考え方

性能評価したい，汎化能力がなるべく高くなるようにハイパーパラメータを決めたい



---
### ホールドアウト

**ホールドアウト**(holdout)は，単純に学習データの一部を取り分けて学習に使わず検証用に使う方法である．

In [ ]:
# 学習データ
N = 25
sig = 0.1
x, y = generateData(N, sigma=sig)
print(x.shape, y.shape)

# うちNL個をあらためて学習用とし，残りを検証用とする
NL = 20
xL, yL = x[:NL], y[:NL]
xV, yV = x[NL:], y[NL:]
NV = len(xV)
print(xL.shape, yL.shape)
print(xV.shape, yV.shape)

In [ ]:
# ホールドアウト法で正則化パラメータを決定するリッジ回帰
D = 9
alphaList = [0.0, 0.00001, 0.0001, 0.001, 0.01] # 正則化パラメータ

msqeL = np.zeros(len(alphaList))
msqeV = np.zeros(len(alphaList))

for i, alpha in enumerate(alphaList):

    # 学習用データを使って学習
    XL = makeDataMatrix(xL, D)
    w = RidgeRegression(XL, yL, alpha)

    # 学習用データに対する msqe
    yL_est = XL @ w
    msqeL[i] = np.mean((yL - yL_est)**2)

    # 検証用データに対する msqe
    XV = makeDataMatrix(xV, D)
    yV_est = XV @ w
    msqeV[i] = np.mean((yV - yV_est)**2)

    print(f'alpha = {alpha:.5f}   msqeL = {msqeL[i]:.5f}   msqeV = {msqeV[i]:.5f}')

# 検証用データに対する msqe が最小のモデルを選ぶ
idx = np.argmin(msqeV)
print()
print(f'The model with alpha = {alphaList[idx]} is chosen.')

---
### クロスバリデーション

ホールドアウト法は簡便だが，元々の学習データ数が少ない場合，検証に使えるデータがごく少数となり，信頼できる検証は難しい．
もうひと手間かける検証法に，**クロスバリデーション**（**交差検証**, **cross validation**）がある．ここでは，クロスバリデーションの方法のひとつである，**n-分割交差検証法** (**n-fold cross validation**)を紹介する．




12個の学習データが与えられたときに 3 分割交差検証を適用する例を説明する．
この場合，学習データを次のように A, B, C の3通りに分ける．

$$
\fbox{A}\fbox{A}\fbox{A}\fbox{A}\fbox{B}\fbox{B}\fbox{B}\fbox{B}\fbox{C}\fbox{C}\fbox{C}\fbox{C}
$$

このとき，A, B, C のうちいずれか1つを検証データにすれば，次の3通りの分け方ができる．

$$
\mbox{学習データ}\fbox{A}\fbox{A}\fbox{A}\fbox{A}\fbox{B}\fbox{B}\fbox{B}\fbox{B}\quad\mbox{検証データ}\fbox{C}\fbox{C}\fbox{C}\fbox{C}\\
\mbox{学習データ}\fbox{A}\fbox{A}\fbox{A}\fbox{A}\fbox{C}\fbox{C}\fbox{C}\fbox{C}\quad\mbox{検証データ}\fbox{B}\fbox{B}\fbox{B}\fbox{B}\\
\mbox{学習データ}\fbox{B}\fbox{B}\fbox{B}\fbox{B}\fbox{C}\fbox{C}\fbox{C}\fbox{C}\quad\mbox{検証データ}\fbox{A}\fbox{A}\fbox{A}\fbox{A}
$$

3通りのそれぞれで学習をおこない，検証の結果を平均する．

---
## scikit-learn

scikit-learn は，Python による機械学習ライブラリの代表格である．


https://scikit-learn.org/

---
### 線形回帰の例

データの準備をしましょう．以前にも使った，小テストの点数と睡眠時間から期末テストの点数を予測する問題を使うことにします．

このデータは，

「Pythonで理解する統計解析の基礎」 谷合廣紀，辻 真吾，技術評論社，2018.

に掲載されているものです．以下の GitHub サイトで公開されています．
https://github.com/ghmagazine/python_stat_sample

In [ ]:
# データの読み込み
df = pd.read_csv('https://github.com/ghmagazine/python_stat_sample/raw/master/data/ch12_scores_reg.csv')
df.drop(columns='通学方法', inplace=True)

# データ配列の作成（定数 1 の列を付け足す必要はない）
XL = df.loc[:, ['小テスト', '睡眠時間']].to_numpy()
yL = df['期末テスト'].to_numpy()
print(XL.shape, yL.shape)

[sklearn.linear_model.LinearRegression](
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) を使って線形回帰の実験をやってみよう．


In [ ]:
from sklearn.linear_model import LinearRegression

# LinearRegression クラスのオブジェクトを生成
#  引数でいろいろ設定できるが，ここではデフォルトで
model = LinearRegression()

# 学習
regressor = model.fit(XL, yL)

# 得られたパラメータを表示
print(f'w_0 = {regressor.intercept_},  [w_1, w_2] = {regressor.coef_}')

In [ ]:
# 適当なテストデータを作って予測させてみる
XT = np.array([[8.0, 10.0], [12.0, 1.0]])
yT_pred = regressor.predict(XT)

for n in range(len(XT)):
    print(f'{XT[n, :]} に対する予測値は {yT_pred[n]:.2f}')

---
### リッジ回帰の例

[sklearn.linear_model.Ridge](
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) でリッジ回帰ができる．

#### 上の例で

上の線形回帰のコードの
```
from sklearn.linear_model import LinearRegression
```
の行を
```
from sklearn.linear_model import Ridge
```
に変え（または追加し）て，
```
model = LinearRegression()
```
の行を
```
model = Ridge(alpha=0.1)
```
に変えれば，簡単にリッジ回帰（$\alpha = 0.1$）の実験をすることができる．やってみよう．

#### 多項式回帰の例で

次は，多項式回帰の例．

In [ ]:
# D次多項式による多項式回帰のための学習データを用意
N = 25
D = 9
x, y = generateData(N, sigma=0.1)
X = np.zeros((N, D)) # 定数 1 の列を付け足す必要はない
for d in range( D):
    X[:, d] = x**d
print(X.shape, y.shape)

[sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) を使うと，ホールドアウト検証が簡単．

In [ ]:
from sklearn.model_selection import train_test_split

# X, y のうち 20 個を学習用に，残りを検証用に，ランダムに分ける
XL, XV, yL, yV = train_test_split(X, y, train_size=20)
print(XL.shape, yL.shape)
print(XV.shape, yV.shape)

In [ ]:
from sklearn.linear_model import Ridge

alphaList = [1e-5, 1e-4, 1e-3, 1e-2, 0.1]

for alpha in alphaList:

    # 学習
    model = Ridge(alpha=alpha)
    regressor = model.fit(XL, yL)

    # 学習データに対する平均二乗誤差
    yL_pred = regressor.predict(XL)
    msqeL = np.mean((yL - yL_pred)**2)

    # 検証データに対する平均二乗誤差
    yV_pred = regressor.predict(XV)
    msqeV = np.mean((yV - yV_pred)**2)

    print(f'alpha = {alpha:.5f}, msqeL = {msqeL:.5f}, msqeV = {msqeV:.5f}')